In [2]:
import json
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [49]:
def getOptimalPath(points_2dList, ep_2dList, maxLinesToEp):
    points_2dList.insert(0, ep_2dList[0]) #ovo nece raditi sa multiple EP sada radi sa jednim
    maxLength = 500000
    minDist = maxLength
    if len(points_2dList)<2:
        raise NameError('Less than 2 points trying to connect')
    for i in range(maxLinesToEp, 0, -1):
        paths, total_distance = getPath(points_2dList, i, maxLength)
        print(f"Testing {i} lines and getting {total_distance} m distance")
        if total_distance <= minDist:
            minDist = total_distance
            minPath = paths
            noOfLines = i
    return minPath

def getPath(points_2dList, noOfLines, maxLength=200000):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(points_2dList, noOfLines)

    # Create the routing index manager.
    #     manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
    #                                            data['num_vehicles'], data['depot'])
    manager = pywrapcp.RoutingIndexManager(len(data['locations']),
                                           data['num_lines'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    distance_matrix = compute_euclidean_distance_matrix(data['locations'])

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        maxLength,  # line maximum distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    #     print(solution)
    # Print solution on console.
    if solution:
        path, total_distance = print_solution(data, manager, routing, solution)

    else:
        print("No solution found!")
    return path, total_distance

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""

    path = []
    max_route_distance = 0
    sum_route_distance = 0
    for vehicle_id in range(data['num_lines']):
        _path = {}
        index = routing.Start(vehicle_id)
        plan_output = 'Route for line {}:\n'.format(vehicle_id)
        _path["line"] = vehicle_id
        route_distance = 0
        _pathL = []
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            _pathL.append(data["locations"][manager.IndexToNode(index)])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        _path["path"] = _pathL
        _path["distance"] = route_distance
        path.append(_path)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        #         print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        sum_route_distance += route_distance
#     print('Maximum of the route distances: {}m'.format(max_route_distance))
#     print('Sum of the route distances: {}m'.format(sum_route_distance))
    return path, sum_route_distance


# [START distance_callback]
def compute_euclidean_distance_matrix(locations):
#     print(f"Locations input for distance calc are: {locations}")
    """Creates callback to return distance between points."""
    distances = {}
#     locations=locations.tolist()
#     print(f"Locations as list for distance calc are: {locations}")
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter or to_counter == 0:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = (int(
                    math.hypot((from_node[0] - to_node[0]),
                               (from_node[1] - to_node[1]))))
    return distances


    # [END distance_callback]
def create_data_model(points_2dList, noOfLines):
    """Stores the data for the problem."""
    data = {}

    data['locations'] = points_2dList
    data['num_lines'] = noOfLines
    data['depot'] = 0
    #     print(data)
    return data




# Start from here

In [52]:

# from flask import Flask, jsonify, request
fname = "testing case (1).json"
fname = "Concept 2.json"
with open(fname, "r") as json_file:
    f=json_file.read()
    data = json.loads(f)
len(data)
#level 1
for i in data:
    print(i)
for i in data["elements"][0]:
    print (i)
#         print(j["name"])

id
name
referenceNumber
description
elements
scenarios
@class
id
name
properties
type
children
template
path
asset
connections
connectionPoints
adjustable
status
creationInfo
scopeOfWorkRelationships


In [55]:
TestSystems = ["Gas", "Oil", "Water"]
for TestSystem in TestSystems:
    cp=[]
    ep=[]
    for element in data["elements"]:
        note=""
        template=element["template"]['name']
        name = element["name"]
    #     print(template)
        for prop in element["properties"]:
            if prop["name"]=="system":
    #             print(prop)
                system=prop["value"]
            if prop["label"]=="Note":
                note = prop['value']
    #             print(note)
            if prop['name']=="position":
                    position = prop["value"]
        if ("Gather" in note) & (system==TestSystem):
            ep.append({"name":name, "position": position})
    #         print(name, note, system)
        if (system==TestSystem) & (template=="OC-xmas"):
            cp.append({"name":name, "position": position})
    allPoints=ep+cp
    print(len(ep), len(cp))
    ep, cp
    ep_2dList = [[pos["position"]["x"],pos["position"]["y"]] for pos in ep]
    cp_2dList = [[pos["position"]["x"],pos["position"]["y"]] for pos in cp]
    ep_2dList, cp_2dList
    # cp_2dList=[[-2456.03, -3802.98], [-1183.03, -5566.78], [-1488.03, -4227.98], [-785.03, -2706.98], [-829.63, -3004.28], [-1197.03, -3081.98], [-1398.03, -3204.98], [-1044.03, -3118.98]]
    # ep_2dList = [[-1655.91, -3139.75]]
    path = getOptimalPath(cp_2dList, ep_2dList, len(cp_2dList))
    path
    pathName=[]
    for line in path:
        _path=[]
        for point in line["path"]:
            for i in allPoints:
    #             print(i["position"]["x"],point[0])
                if (i["position"]["x"]==point[0]) & (i["position"]["y"]==point[1]):
                    _path.append(i["name"])
        pathName.append(_path)
    print(pathName)

1 8
Testing 8 lines and getting 7042 m distance
Testing 7 lines and getting 7042 m distance
Testing 6 lines and getting 7042 m distance
Testing 5 lines and getting 7042 m distance
Testing 4 lines and getting 7042 m distance
Testing 3 lines and getting 5285 m distance
Testing 2 lines and getting 6930 m distance
Testing 1 lines and getting 5106 m distance
[['Gas Manifold New', 'Gas Well 2', 'Gas Well 3', 'Gas Well 1', 'Gas Well 7', 'Gas Well 6', 'Gas Well 8', 'Gas Well 5', 'Gas Well 4']]
1 4
Testing 4 lines and getting 2217 m distance
Testing 3 lines and getting 2217 m distance
Testing 2 lines and getting 2259 m distance
Testing 1 lines and getting 1539 m distance
[['Oil Manifold New', 'Oil Well 4', 'Oil Well 3', 'Oil Well 1', 'Oil Well 2']]
1 3
Testing 3 lines and getting 4718 m distance
Testing 2 lines and getting 4913 m distance
Testing 1 lines and getting 4642 m distance
[['Water Manifold New', 'Water Well 1', 'Water Well 2', 'Water Well 3']]


In [47]:
j = json.dumps({"ep":ep,"cp":cp})
j

'{"ep": [{"name": "Gas Manifold New", "position": {"x": -1104.9998, "y": -5046.074, "z": -1420.8833}}], "cp": [{"name": "Gas Well 7", "position": {"x": -1398.03, "y": -3204.98, "z": -1471.5887}}, {"name": "Gas Well 6", "position": {"x": -1197.03, "y": -3081.98, "z": -1474.8545}}, {"name": "Gas Well 8", "position": {"x": -1044.03, "y": -3118.98, "z": -1469.5128}}, {"name": "Gas Well 1", "position": {"x": -2456.03, "y": -3802.98, "z": -1474.5887}}, {"name": "Gas Well 2", "position": {"x": -1183.03, "y": -5566.78, "z": -1401.6305}}, {"name": "Gas Well 3", "position": {"x": -1488.03, "y": -4227.98, "z": -1442.5248}}, {"name": "Gas Well 4", "position": {"x": -785.03, "y": -2706.98, "z": -1473.5082}}, {"name": "Gas Well 5", "position": {"x": -829.63, "y": -3004.28, "z": -1471.9469}}]}'